# <h3 align="center"> LinearRegressionOnSuicideDataFromKaggle </h3>

### Before answering any questions, I wanted to import any required libraries, and look at the data. Also, I wanted to do some pleminary preprocessing so I could have some context before answering. 

#### I randomly selected a subset of data, and applied different models to it to see how differing methods would show different results. I picked age 20, male, and generation X

In [1]:

import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)

In [2]:
df = pd.read_csv('master.csv')


In [3]:
#Want to look at the data
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [4]:
# Check for duplicates
df["is_duplicate"]= df.duplicated()

# Check number of duplicates
print(f"#total= {len(df)}")
print(f"#duplicated= {len(df[df['is_duplicate']==True])}")

#total= 27820
#duplicated= 0


There are no duplicates found, so we can move on.

First, in my experience, coding langauges do not do well with spaces as characters, so I will rename some columns, like HDI for year.

In [5]:
#renaming columns
df.rename(columns={"HDI for year":"yearly_HDI"}, inplace=True)
df.rename(columns={" gdp_for_year ($) ":"gdp"}, inplace=True)
# Do we have NaN in our dataset?
df.isnull().any()

country               False
year                  False
sex                   False
age                   False
suicides_no           False
population            False
suicides/100k pop     False
country-year          False
yearly_HDI             True
gdp                   False
gdp_per_capita ($)    False
generation            False
is_duplicate          False
dtype: bool

In [6]:
x = df['yearly_HDI'].isnull().sum()
y = len(df['yearly_HDI'])
print ('There are',x,'entries with NaN in yearly_HDI column')
print ('That means,',100*x/y,'% of entries are NaN values in this column.')

There are 19456 entries with NaN in yearly_HDI column
That means, 69.9352983465133 % of entries are NaN values in this column.


In [7]:
df['yearly_HDI'] = df['yearly_HDI'].fillna(df['yearly_HDI'].mean())

In [8]:
cols = ['country-year','gdp','is_duplicate','suicides_no','country','population','year'] #dropping country and gdp according to Guven's office hour recording
df=df.drop(cols,axis='columns')


In [9]:
#To check missing nominal values, lets use a loop function
for col in df.columns:
    if df[col].dtype == object:
        print(col, df[col].unique())

sex ['male' 'female']
age ['15-24 years' '35-54 years' '75+ years' '25-34 years' '55-74 years'
 '5-14 years']
generation ['Generation X' 'Silent' 'G.I. Generation' 'Boomers' 'Millenials'
 'Generation Z']


Luckily, there are no '?' so nothing needs to be imputed

In [10]:
# Print nominal variables
for f in list(df.columns.values):
    if df[f].dtype == object:
        print(f) 

sex
age
generation


In [11]:
#Sanity Check
df.head()

,sex,age,suicides/100k pop,yearly_HDI,gdp_per_capita ($),generation
0,male,15-24 years,6.71,0.776601,796,Generation X
1,male,35-54 years,5.19,0.776601,796,Silent
2,female,15-24 years,4.83,0.776601,796,Generation X
3,male,75+ years,4.59,0.776601,796,G.I. Generation
4,male,25-34 years,3.28,0.776601,796,Boomers


Separating the dependent variable, and the rest, while also one hot encoding the remaining columns

In [12]:
y = df.pop('suicides/100k pop')

In [13]:
X = pd.get_dummies(df)
X

,yearly_HDI,gdp_per_capita ($),sex_female,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,0.776601,796,0,1,1,0,0,0,0,0,0,0,1,0,0,0
1,0.776601,796,0,1,0,0,1,0,0,0,0,0,0,0,0,1
2,0.776601,796,1,0,1,0,0,0,0,0,0,0,1,0,0,0
3,0.776601,796,0,1,0,0,0,0,0,1,0,1,0,0,0,0
4,0.776601,796,0,1,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27815,0.675000,2309,1,0,0,0,1,0,0,0,0,0,1,0,0,0
27816,0.675000,2309,1,0,0,0,0,0,0,1,0,0,0,0,0,1
27817,0.675000,2309,0,1,0,0,0,1,0,0,0,0,0,1,0,0
27818,0.675000,2309,1,0,0,0,0,1,0,0,0,0,0,1,0,0


In [14]:
y

0        6.71
1        5.19
2        4.83
3        4.59
4        3.28
         ... 
27815    2.96
27816    2.58
27817    2.17
27818    1.67
27819    1.46
Name: suicides/100k pop, Length: 27820, dtype: float64

In [15]:
from sklearn.linear_model import LinearRegression
lg = LinearRegression()
lg.fit(X,y)
lg.score(X,y)

0.29770795573923414

In [16]:

import numpy as np

X_1 = np.where((X['age_15-24 years'] == 1) & (X['sex_male'] == 1) & (X['generation_Generation X'] == 1))
X_1

(array([    0,    13,    28, ..., 27656, 27667, 27679]),)

In [17]:
df_loc = X.loc[X_1]
df_loc

,yearly_HDI,gdp_per_capita ($),sex_female,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,0.776601,796,0,1,1,0,0,0,0,0,0,0,1,0,0,0
13,0.776601,769,0,1,1,0,0,0,0,0,0,0,1,0,0,0
28,0.776601,833,0,1,1,0,0,0,0,0,0,0,1,0,0,0
37,0.776601,251,0,1,1,0,0,0,0,0,0,0,1,0,0,0
48,0.776601,437,0,1,1,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27632,0.776601,703,0,1,1,0,0,0,0,0,0,0,1,0,0,0
27644,0.776601,724,0,1,1,0,0,0,0,0,0,0,1,0,0,0
27656,0.776601,719,0,1,1,0,0,0,0,0,0,0,1,0,0,0
27667,0.776601,801,0,1,1,0,0,0,0,0,0,0,1,0,0,0


In [18]:
mean_HDI = df_loc['yearly_HDI'].mean()
mean_GDP = df_loc['gdp_per_capita ($)'].mean()
pred = np.array([mean_HDI, mean_GDP, 0, 1, 1, 0, 0,0,0,0,0,0,1,0,0,0])
pred.shape

(16,)

In [19]:
pred = pred.reshape(1,16)
pd.DataFrame(pred, columns=X.columns)

,yearly_HDI,gdp_per_capita ($),sex_female,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,0.76692,10910.432356,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
y_true = y.loc[X_1]
y_true.mean()

14.512961210974456

In [21]:
y_true = y_true.mean().ravel()
y_true

array([14.51296121])

In [22]:
y_pred = lg.predict(pred)
y_pred.shape

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(1,)

In [23]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred)

2.426010089130198

In [24]:
print('The number of regression coefficients is',len(lg.coef_))

The number of regression coefficients is 16


# Predicting it without one hot encoding, and manually replacinng the nominal variables with numbers

In [25]:
df2 = df.copy()

In [26]:
df2['sex'] = df['sex'].replace({
 'female':0,
 'male':1})
df2['age'] = df['age'].replace({
 '5-14 years':10,
 '15-24 years':20,
 '25-34 years':30,
 '35-54 years':45,
 '55-74 years':65,
 '75+ years':80})
df2['generation'] = df['generation'].replace({
 'G.I. Generation':1,
 'Silent':2,
 'Boomers':3,
 'Generation X':4,
 'Millenials':5,
 'Generation Z':6})
df2.head()

,sex,age,yearly_HDI,gdp_per_capita ($),generation
0,1,20,0.776601,796,4
1,1,45,0.776601,796,2
2,0,20,0.776601,796,4
3,1,80,0.776601,796,1
4,1,30,0.776601,796,3


In [27]:
pred_new = np.array([1,20, mean_HDI, mean_GDP,4])
pred_new.shape

(5,)

In [28]:
pred_new = pred_new.reshape(1,5)
pd.DataFrame(pred_new, columns=df2.columns)

,sex,age,yearly_HDI,gdp_per_capita ($),generation
0,1.0,20.0,0.76692,10910.432356,4.0


In [29]:
lg.fit(df2,y)
lg.score(df2,y)

0.28117332148356955

In [30]:
y_pred_new = lg.predict(pred_new)
y_pred_new.shape

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(1,)

In [31]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred_new)

0.40082336511560257

In [32]:
print (f'line coefficients are {lg.coef_.shape}')

line coefficients are (5,)


# Some thoughs 
The second number is much better, in terms of MAE accuracy, as seen above. Ordinal gives it relationships between age, one hot encoding cannot do that. There is a difference in performance between the two models. The MAE error of prediction in the first model, which used one-hot encoded variables with multiple linear regression, is lower (i.e., better) than in the second model, which used numerical labels with multiple linear regression. This implies that the first model fits the data better and has better predictive power.

Furthermore, the first model with one-hot encoded variables has 16 regression coefficients, whereas the second model only has 5. Because we converted two categorical variables (age and generation) into numerical labels in the second model, the number of independent variables was reduced. Reducing the number of variables had no effect on the model's performance.

#### Now to test it with a different parameter,age 33, male and generation Alpha (i.e. the generation after generation Z)

In [33]:
pred3 = np.array([1,33, mean_HDI, mean_GDP,7])
pred3 = pred3.reshape(1,5)

In [34]:
pred3 = lg.predict(pred3)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [35]:
pred3


array([13.63663109])

# One advantage when using regression (as opposed to classification with nominal features) in terms of independent variables
In simple words, it gives nuance, vs just 'low high medium' qualitative factors. Quantitative predictions vs qualitative is always better.Using regression in terms of independent variables (as opposed to classification with nominal features) is that it allows us to use continuous variables as independent variables. This means that we can use variables with a range of values as inputs to the model, such as age, income, or temperature. In contrast, we are limited to using discrete variables, such as gender or color, with nominal features, which may not provide as much information for the model to make predictions.

# One advantage when using regular numerical values rather than one-hot encoding for regression

Reduces the dimensionality of the data. One-hot encoding generates new variables for each category of a categorical variable, resulting in a large number of variables and making the model more complex and difficult to interpret. Using numerical values, on the other hand, reduces the number of variables and can make the model simpler and more interpretable. Moreover, the data gets more context.


# The method which I think is better

Based on my findings, I would recommend using the regression model rather than the classifier for the given problem. This is due to the fact that we are forecasting a continuous variable (the suicide rate) rather than a categorical variable. Furthermore, in terms of MAE error, the regression model with one-hot encoded variables outperformed the classifier. As a result, we can anticipate that the regression model will provide better predictions for this problem.